In [1]:
# State

In [2]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_z0_1q, get_z1_1q, get_x0_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
start_all = time.time()

In [5]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
tester_objects = [povm_x, povm_y, povm_z]

In [13]:
# Case 1:
# true_object = get_z0_1q(c_sys)
# true_object.vec

# Case 2:
# vec = np.array([1/np.sqrt(2), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6)],dtype=np.float64)
# true_object = State(c_sys, vec)

# Case 3:
vec = np.array([1 / np.sqrt(2), 0, 0, 0], dtype=np.float64)
true_object = State(c_sys, vec)

In [11]:
num_data = [100, 1000, 10000]
n_rep = 100

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
]

seed = 777
qtomography_list = [
    StandardQst(tester_objects, on_para_eq_constraint=True, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=False, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=True, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=False, seed=seed),
]
para_list = [True, False, True, False]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
]

estimation_results_list = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]
    
    start = time.time()
    print(f"Case {i}: {name}")
    print(f"Parametorization: {para_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.estimate(
       qtomography=qtomography,
       true_object=true_object,
       num_data=num_data,
       estimator=estimator,
       iteration=n_rep,
    )
    estimation_results_list.append(estimation_results)
    
    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")

  0%|          | 0/100 [00:00<?, ?it/s]

Case 0: LinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQst
Estimator: LinearEstimator


  1%|          | 1/100 [00:00<00:10,  9.18it/s]

elapsed_time:0.1666849652926127[min]

Case 1: LinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQst
Estimator: LinearEstimator


  0%|          | 0/100 [00:00<?, ?it/s]/Users/tomoko/project/rcast/workspace/quara/quara/objects/state.py:283: ComplexWarning:

Casting complex values to real discards the imaginary part

  1%|          | 1/100 [00:00<00:15,  6.60it/s]

elapsed_time:0.1668148676554362[min]

Case 2: ProjectedLinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQst
Estimator: ProjectedLinearEstimator


  1%|          | 1/100 [00:00<00:14,  7.07it/s]

elapsed_time:0.22587915261586508[min]

Case 3: ProjectedLinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQst
Estimator: ProjectedLinearEstimator


100%|██████████| 100/100 [00:13<00:00,  7.39it/s]

elapsed_time:0.22597064971923828[min]



In [12]:
report.export_report("sample_modified_empi_dists_mse_case02_全部の軸から一番離れた表面.pdf",
    estimation_results_list,
    case_name_list,
    qtomography_list,
    para_list,
    estimator_list,
    true_object,
    tester_objects,
    num_data,
    n_rep,
    seed=seed,
)

100%|██████████| 100/100 [00:00<00:00, 64857.03it/s]


calc_mse_prob_dists: 0.01065637918734454
calc_mse_prob_dists: 0.0011260169667668836
calc_mse_prob_dists: 8.372444444782063e-05


/Users/tomoko/project/rcast/workspace/quara/venv/lib/python3.8/site-packages/numpy/core/_methods.py:216: RuntimeWarning:

Degrees of freedom <= 0 for slice

/Users/tomoko/project/rcast/workspace/quara/venv/lib/python3.8/site-packages/numpy/core/_methods.py:209: RuntimeWarning:

invalid value encountered in double_scalars

/Users/tomoko/project/rcast/workspace/quara/quara/objects/state.py:283: ComplexWarning:

Casting complex values to real discards the imaginary part



Completed to export pdf. (sample_modified_empi_dists_mse_case02_全部の軸から一番離れた表面.pdf)


# Debug

In [128]:
true_object = get_z0_1q(c_sys)
num_data = [100, 1000]
n_rep = 100
seed_list = [100, 101, 102, 103, 104]

case_name_list = [f"seed={seed}" for seed in seed_list]


qtomography_list = [
    StandardQst(tester_objects, on_para_eq_constraint=True, seed=seed)
    for seed in seed_list
]
para_list = [True] * len(qtomography_list)

estimator_list = [LinearEstimator()] * len(qtomography_list)

estimation_results_list = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]

    start = time.time()
    print(f"Case {i}: {name}")
    print(f"Parametorization: {para_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.estimate(
        qtomography=qtomography,
        true_object=true_object,
        num_data=num_data,
        estimator=estimator,
        iteration=n_rep,
    )
    estimation_results_list.append(estimation_results)

    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")

  4%|▍         | 4/100 [00:00<00:02, 33.57it/s]

Case 0: seed=100
Parametorization: True
Type of qtomography: StandardQst
Estimator: LinearEstimator
_estimate: [[(100, array([0.56, 0.44])), (100, array([0.45, 0.55])), (100, array([1., 0.]))], [(1000, array([0.505, 0.495])), (1000, array([0.499, 0.501])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.49, 0.51])), (100, array([0.45, 0.55])), (100, array([1., 0.]))], [(1000, array([0.521, 0.479])), (1000, array([0.499, 0.501])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.55, 0.45])), (100, array([0.46, 0.54])), (100, array([1., 0.]))], [(1000, array([0.516, 0.484])), (1000, array([0.485, 0.515])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.49, 0.51])), (100, array([0.52, 0.48])), (100, array([1., 0.]))], [(1000, array([0.471, 0.529])), (1000, array([0.514, 0.486])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.45, 0.55])), (100, array([0.54, 0.46])), (100, array([1., 0.]))], [(1000, array([0.485, 0.515])), (1000, array([0.486, 0.514])), (1000, arra

 16%|█▌        | 16/100 [00:00<00:01, 42.45it/s]

_estimate: [[(100, array([0.51, 0.49])), (100, array([0.54, 0.46])), (100, array([1., 0.]))], [(1000, array([0.522, 0.478])), (1000, array([0.509, 0.491])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.49, 0.51])), (100, array([0.66, 0.34])), (100, array([1., 0.]))], [(1000, array([0.506, 0.494])), (1000, array([0.506, 0.494])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.5, 0.5])), (100, array([0.5, 0.5])), (100, array([1., 0.]))], [(1000, array([0.502, 0.498])), (1000, array([0.494, 0.506])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.49, 0.51])), (100, array([0.59, 0.41])), (100, array([1., 0.]))], [(1000, array([0.537, 0.463])), (1000, array([0.526, 0.474])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.53, 0.47])), (100, array([0.51, 0.49])), (100, array([1., 0.]))], [(1000, array([0.519, 0.481])), (1000, array([0.479, 0.521])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.51, 0.49])), (100, array([0.54, 0.46])), (100, array([1., 0.]

 28%|██▊       | 28/100 [00:00<00:01, 46.69it/s]

...

_estimate: [[(100, array([0.43, 0.57])), (100, array([0.66, 0.34])), (100, array([1., 0.]))], [(1000, array([0.478, 0.522])), (1000, array([0.527, 0.473])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.46, 0.54])), (100, array([0.5, 0.5])), (100, array([1., 0.]))], [(1000, array([0.526, 0.474])), (1000, array([0.511, 0.489])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.43, 0.57])), (100, array([0.47, 0.53])), (100, array([1., 0.]))], [(1000, array([0.504, 0.496])), (1000, array([0.491, 0.509])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.5, 0.5])), (100, array([0.52, 0.48])), (100, array([1., 0.]))], [(1000, array([0.498, 0.502])), (1000, array([0.496, 0.504])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.53, 0.47])), (100, array([0.5, 0.5])), (100, array([1., 0.]))], [(1000, array([0.501, 0.499])), (1000, array([0.502, 0.498])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.49, 0.51])), (100, array([0.49, 0.51])), (100, array([1., 0.]))

 87%|████████▋ | 87/100 [00:01<00:00, 64.69it/s]

_estimate: [[(100, array([0.43, 0.57])), (100, array([0.44, 0.56])), (100, array([1., 0.]))], [(1000, array([0.516, 0.484])), (1000, array([0.523, 0.477])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.46, 0.54])), (100, array([0.5, 0.5])), (100, array([1., 0.]))], [(1000, array([0.509, 0.491])), (1000, array([0.505, 0.495])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.48, 0.52])), (100, array([0.52, 0.48])), (100, array([1., 0.]))], [(1000, array([0.502, 0.498])), (1000, array([0.507, 0.493])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.44, 0.56])), (100, array([0.51, 0.49])), (100, array([1., 0.]))], [(1000, array([0.507, 0.493])), (1000, array([0.499, 0.501])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.47, 0.53])), (100, array([0.62, 0.38])), (100, array([1., 0.]))], [(1000, array([0.504, 0.496])), (1000, array([0.498, 0.502])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.46, 0.54])), (100, array([0.46, 0.54])), (100, array([1., 0

100%|██████████| 100/100 [00:01<00:00, 61.90it/s]

_estimate: [[(100, array([0.51, 0.49])), (100, array([0.45, 0.55])), (100, array([1., 0.]))], [(1000, array([0.501, 0.499])), (1000, array([0.521, 0.479])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.55, 0.45])), (100, array([0.56, 0.44])), (100, array([1., 0.]))], [(1000, array([0.505, 0.495])), (1000, array([0.495, 0.505])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.48, 0.52])), (100, array([0.44, 0.56])), (100, array([1., 0.]))], [(1000, array([0.503, 0.497])), (1000, array([0.492, 0.508])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.55, 0.45])), (100, array([0.45, 0.55])), (100, array([1., 0.]))], [(1000, array([0.497, 0.503])), (1000, array([0.485, 0.515])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.55, 0.45])), (100, array([0.5, 0.5])), (100, array([1., 0.]))], [(1000, array([0.503, 0.497])), (1000, array([0.483, 0.517])), (1000, array([1., 0.]))]]
_estimate: [[(100, array([0.5, 0.5])), (100, array([0.45, 0.55])), (100, array([1., 0.]

In [129]:
data_analysis.make_empi_dists_mse_graph_for_debug(
    estimation_results_list,
    qtomography_list,
    true_object,
    num_data,
    n_rep,
    tester_objects,
    case_name_list,
)

100%|██████████| 100/100 [00:00<00:00, 31467.51it/s]

calc_mse_prob_dists: 0.009932000000000003
calc_mse_prob_dists: 0.0012387600000000017
calc_mse_prob_dists: 0.01093
calc_mse_prob_dists: 0.0007758800000000013



100%|██████████| 100/100 [00:00<00:00, 52494.42it/s]


calc_mse_prob_dists: 0.010766000000000006
calc_mse_prob_dists: 0.001101440000000002
calc_mse_prob_dists: 0.010675999999999996


100%|██████████| 100/100 [00:00<00:00, 11060.93it/s]


calc_mse_prob_dists: 0.0010529600000000012
calc_mse_prob_dists: 0.011067999999999998
calc_mse_prob_dists: 0.0009100400000000012
call: calc_mse_empi_dists_analytical
call: calc_mse_empi_dists_analytical
